### IS Proof of concept
Finding Product position on Social Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [5]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [8]:
from IPython.display import clear_output

In [9]:
# Files definition
root_path = 'Datasource'

comment_file = []
comment_file.append(root_path+"/comment_meiji.csv")
comment_file.append(root_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_path+"/comment_nlpToken.csv"
comment_cooc_nlp_file = root_path+"/comment_cooc.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [10]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]

In [11]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    stopwords.remove('ไม่')
    stopwords.remove('ได้')
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [12]:
def checkThai(txt):
    prop = pythainlp.util.countthai(txt)
    return prop

In [13]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i).lower())
    return returnList

In [14]:
def tagThai(w):
    pd_returnList = pd.DataFrame(pos_tag(w, corpus='pud'))
    returnList = pd_returnList.iloc[:,1]
    return returnList

In [15]:
def soundThai(w):
    returnList = []
    for i in w:
        returnList.append(romanize(i, engine="thai2rom"))
    return returnList

In [16]:
list_punc = []
for c in string.punctuation:
    list_punc.append(c)

### Clean data & NLP Tokenized

In [17]:
df_textList1 = pd.read_csv(comment_file[0],header=None)
df_textList2 = pd.read_csv(comment_file[1],header=None)

In [18]:
df_textList1.insert(0,'Table','Meiji')
df_textList1.columns = ['Table','URLs', 'text', 'Retrived_date']
df_textList2.insert(0,'Table','DDFC')
df_textList2.columns = ['Table','URLs', 'text', 'Retrived_date']

In [19]:
pd.DataFrame(df_textList1.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,1000.0,43.049,53.682105,3.0,11.0,21.0,49.0,351.0


In [20]:
print('meiji comment:',df_textList1.shape[0])
print('meiji URLs ก่อนตัด:',len(df_textList1.URLs.unique()))

meiji comment: 43049
meiji URLs ก่อนตัด: 1000


In [21]:
pd.DataFrame(df_textList2.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,517.0,44.756286,57.056773,1.0,10.0,19.0,53.0,328.0


In [22]:
print('DDCF comment:',df_textList2.shape[0])
print('DDCF URLs ก่อนตัด:',len(df_textList2.URLs.unique()))

DDCF comment: 23139
DDCF URLs ก่อนตัด: 517


In [23]:
df_Corpus = pd.concat([df_textList1,df_textList2],ignore_index=True)

In [24]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

All comment: 66188
All URLs ก่อนตัด: 1517


In [25]:
missing = df_Corpus.isnull().sum()
print(missing)

Table               0
URLs                0
text             4703
Retrived_date       0
dtype: int64


In [26]:
df_Corpus.tail()

,Table,URLs,text,Retrived_date
66183,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [27]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','Table','URLs', 'text', 'Retrived_date']

In [28]:
df_Corpus.tail()

,commentId,Table,URLs,text,Retrived_date
66183,66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,66188,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [29]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [30]:
missing = df_Corpus.isnull().sum()
print(missing)

Table            0
URLs             0
text             0
Retrived_date    0
dtype: int64


In [31]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [32]:
df_Corpus.insert(4,'token_text',None)
#df_Corpus.insert(5,'token_sound',None)
#df_Corpus.insert(6,'token_tag',None)
df_Corpus.tail()

,Table,URLs,text,Retrived_date,token_text
commentId,,,,,
66181,DDFC,https://pantip.com/topic/30651064,นมผง คือเอานมดิบดีๆ มาคัดไขมันนมและอื่นๆออกไปใ...,2020-10-27 13:54:33.920930,None
66182,DDFC,https://pantip.com/topic/30651064,นมสด รสชาติมันดีนะ\nเพราะมีปริมาณไขมันสูง ดังน...,2020-10-27 13:54:33.920930,None
66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930,None
66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930,None
66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930,None


เก็บเฉพาะหัวกระทู้

In [33]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','text']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_text',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

,URLs,text,token_text,mention_product,mention_brand
commentId,,,,,
65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,None,None,None
66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,None,None,None
66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...",None,None,None
66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,None,None,None
66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,None,None,None


## ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [34]:
garbage_char = ['',' ','🥺','\r','\n','\r\n','\xa0','www','application','ทางกฎหมาย','html'
                ,'http','https','url','com','pantip','topic','_','–','พันทิป','^^'
                ,'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ','แฮ่','เย่ๆๆ','ว๊าว','ฮิ้วววววววววว','Number','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','(',')','ฯ','ๆ','กก','อันนี้','%s','😆', '🤣','😢'
                ,'😏','😂','😿','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','://','มกราคม', 'กุมภาพันธ์'
                ,'มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม', 'สิงหาคม','กันยายน', 'ตุลาคม'
                ,'พฤศจิกายน', 'ธันวาคม']
garbage_char  = garbage_char + list_punc

In [35]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [36]:
custom_list = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','betagen','บีทาเก้น','บีทาเกน'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โชคชัย','แดรี่โฮม','dairy home','เดรี่โฮม','อืมม มิลค์','Umm! Milk','เอ็มมิลค์'
               ,'mmilk','เอ็มมิ้ลค์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น'
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม','high protein'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์'
               ,'เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิ บัลแกเรีย','เมจิบัลแกเรีย','นมเปรี้ยว','รสธรรมชาติ','รสกลมกล่อม','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม'
               ,'น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่','ดาร์คช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน'
               ,'ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','แมคโคร','แม็คโคร','โลตัส','บิ๊กซี','bigc','Golden place','Big C'
               ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม'
               ,'แพ้นม','แพ้นมวัว','เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก'
               ,'คีเฟอร์นม','พันทิป','ร้านนม','ซื้อไม่ได้','ไม่ได้','ลูกสุนัข','ลูกหมา','คายทิ้ง','เจมส์ จิ','เจมส์จิ'
              ]
words = set(thai_words()).union(set(custom_list))
custom_tokenizer = Tokenizer(engine='newmm', custom_dict=words, keep_whitespace=False)

In [37]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โชคชัย','แดรี่โฮม','dairy home','Dairy Home','เดรี่โฮม','อืมม มิลค์','Umm! Milk','เอ็มมิลค์'
               ,'mmilk','เอ็มมิ้ลค์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น']
    products = ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','นม'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม','high protein'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์','เมจิโกลด์ แม็กซ์'
               ,'น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่','ดาร์คช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน','เมจิโกลด์ แม็กซ์','เมจิโกลด์'
               ,'เมจิ บัลแกเรีย','เมจิบัลแกเรีย','นมเปรี้ยว','รสธรรมชาติ','รสกลมกล่อม','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม','ซากุระ']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    else:
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

In [38]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "text": df.text[c],
            "token_text":df.token_text[c],
            "mention_product": df.mention_product[c],
            "mention_brand": df.mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [39]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "Table": df.Table[c],
            "URLs": lx,
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c]
        }
        listofdict.append(info)
    return listofdict

In [40]:
def tokenize_custom(x,custom_tokenizer):
    wtkn = custom_tokenizer.word_tokenize(x)
    wtkn = checkStopwords(wtkn)
    wtkn = [x for x in wtkn if len(x) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [41]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','meiji','meji','miji']:
            ret_wtkn[i] = 'เมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','foremost']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['แดรี่โฮม','dairy home','เดรี่โฮม']:
            ret_wtkn[i] = 'แดรี่โฮม'
        elif each  in ['โชคชัย','อืมม มิลค์','umm milk','นมฟาร์มโชคชัย']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0%','0% fat','ขาดมันเนย']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','ไฮโปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'lotus'
        elif each  in ['บิ๊กซี','bigc','Big C']:
            ret_wtkn[i] = 'bigc'
        elif each  in ['ท้องเสีย','ขี้แตก']:
            ret_wtkn[i] = 'ท้องเสีย'
        elif each  in ['ลูกสุนัข','ลูกหมา']:
            ret_wtkn[i] = 'ลูกสุนัข'
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [ ]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

1.Tokenize comment

In [ ]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x,custom_tokenizer))

In [44]:
df_process_comment.head()

,commentId,Table,URLs,text,Retrived_date,token_text
0,1,Meiji,https://pantip.com/topic/39868603,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู..."
1,2,Meiji,https://pantip.com/topic/39868603,เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน]
2,3,Meiji,https://pantip.com/topic/39868603,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-10-26 12:40:26.135083,"[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม..."
3,4,Meiji,https://pantip.com/topic/39868603,เป็นรสที่ขายไม่ดีครับ\r\nรสพื้นฐานคือ\r\nจืด แ...,2020-10-26 12:40:26.135083,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, %,, low..."
4,5,Meiji,https://pantip.com/topic/39868603,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-10-26 12:40:26.135083,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่..."


2.Generalized keyword (Brand & product)

In [45]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

3.Insert comments to DB

In [46]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

4.Tokenize head of thread

In [47]:
df_process_thread['token_text'] = df_process_thread['text'].apply(lambda x: tokenize_custom(x,custom_tokenizer))

5.Generalized keyword (Brand & product)

In [48]:
df_process_thread['token_text'] = df_process_thread['token_text'].apply(lambda x: generalize_keyword(x))

6.Mark domain/entity mention

In [49]:
df_process_thread['mention_brand'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['mention_product'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'p'))

In [50]:
df_process_thread.tail()

,commentId,URLs,text,token_text,mention_product,mention_brand
1509,65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,"[คุณแม่มือใหม่, ทำตัว, ไม่, ทราบ, ที่จะ, กินนม...",[],[]
1510,66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,"[พรุ่งนี้, ลูกชาย, อายุ, ขวบ, เดือน, หม่ำ, นม,...",[นม],[]
1511,66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...","[เพื่อน, คุ้นเคย, คำถาม, ทุ, กค, น., .., ?"", ห...",[],[]
1512,66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,"[แตก, กระทู้, มาจาก, บ้าน, รูป, เฟส, นะจ๊ะ, วั...",[],[]
1513,66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,"[อยากรู้, นมกล่อง, ทึ่, ขาย, ห้างสรรพสินค้า, น...",[นม],[]


7.Insert thread to DB

In [51]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [52]:
del df_textList1,df_textList2,df_thread,df_Corpus,df_process_comment,df_process_thread

<br><br>
### Process หัวกระทู้เพื่อดึง Domain (Brand & Product) ไม่ใช้โมเดลเทรนแล้วทำนาย ต้องการความถูกต้อง 100%

หากเคยบันทึกลง MongoDB แล้ว ให้เริ่มตรงนี้

In [19]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [20]:
df_urls_milk = df_thr_process[(df_thr_process.mention_brand.str.len() != 0) & (df_thr_process.mention_product.str.len() != 0)]
df_urls_milk['mention_brand'] = df_urls_milk['mention_brand'].apply(lambda x: repr(set(x)))
df_urls_milk['mention_product'] = df_urls_milk['mention_product'].apply(lambda x: repr(set(x)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
df_urls_milk.shape

(422, 6)

In [22]:
df_view = df_urls_milk[['URLs','mention_brand']]
df_view.column = ['URLs','mention_brand']
df_view_count = df_view.groupby('mention_brand').count()
print('จำนวนกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ในหัวกระทู้ =',df_view.shape[0])

จำนวนกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ในหัวกระทู้ = 422


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [23]:
df_view_count.sort_values(by='URLs',ascending=False)

,URLs
mention_brand,
{'เมจิ'},188
{'โฟร์โมสต์'},69
{'ดัชมิลล์'},51
{'โชคชัย'},24
"{'เมจิ', 'ดัชมิลล์'}",13
{'ไทยเดนมาร์ค'},8
"{'เมจิ', 'คาเนชั่น'}",6
"{'เมจิ', 'โฟร์โมสต์'}",6
"{'เมจิ', 'โชคชัย'}",3


In [24]:
df_urls_milk[df_urls_milk['mention_brand']=="{'เมจิ'}"]

,URLs,_id,mention_brand,mention_product,text,token_text
0,https://pantip.com/topic/39868603,5fbb5a418e8105c375c6e325,{'เมจิ'},"{'นม', 'รสหวาน'}",เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู..."
1,https://pantip.com/topic/39597709,5fbb5a418e8105c375c6e326,{'เมจิ'},{'นม'},ตามหัวข้อเลยค่ะ อยากรู้ว่านมเมจิที่ต้มก่อนนำไป...,"[หัวข้อ, อยากรู้, นม, เมจิ, ต้ม, ส่วนผสม, เครื..."
2,https://pantip.com/topic/39747201,5fbb5a418e8105c375c6e327,{'เมจิ'},{'นม'},วันก่อนซื้อ นมเมจิขวด 2 ลิตร มากิน สังเกตฝามัน...,"[วันก่อน, ซื้อ, นม, เมจิ, ขวด, ลิตร, กิน, สังเ..."
3,https://pantip.com/topic/39732615,5fbb5a418e8105c375c6e328,{'เมจิ'},{'นม'},อยากรู้ว่าMakroมีนมยี่ห้อ meiji(เมจิ) ขายมั้ยครับ,"[อยากรู้, makro, นม, ยี่ห้อ, เมจิ, เมจิ, ขาย]"
5,https://pantip.com/topic/39928742,5fbb5a418e8105c375c6e32a,{'เมจิ'},"{'นม', 'ไขมัน 0%'}",สวัสดี เราคือมนุษย์ออฟฟิศ อายุ 25 ปี ที่เสพติด...,"[สวัสดี, มนุษย์, ออฟฟิศ, อายุ, ปี, เสพติด, กิน..."
10,https://pantip.com/topic/39975832,5fbb5a418e8105c375c6e32f,{'เมจิ'},{'นม'},🥺 คือผมอายุ17ย่าง18 สูง168หนัก61 ผมไม่สูงขึ้นเ...,"[ผม, อายุ, ย่าง, หนัก, ผม, ไม่, อายุ, เพื่อน, ..."
11,https://pantip.com/topic/39905450,5fbb5a418e8105c375c6e330,{'เมจิ'},{'นม'},ขอคำแนะนำหน่อยครับ \r\nทำสปาเกตตี้คาโบนาราครั้...,"[คำแนะนำ, ทำ, สปา, เก, ตตี้, คา, โบ, นารา, ครั..."
12,https://pantip.com/topic/39874488,5fbb5a418e8105c375c6e331,{'เมจิ'},{'นม'},พอดีว่าช่วงนี้ WFH เลยอยากจะลองชง มอคค่า เองบ้...,"[wfh, ลอง, ชง, มอคค่า, ลอง, ทำ, เวิค, รสชาติ, ..."
17,https://pantip.com/topic/39099897,5fbb5a418e8105c375c6e336,{'เมจิ'},"{'นม', 'พาสเจอร์ไรส์'}",พอดีวันนี้ไปซื้อนมเมจิ พาสเจอไรซ์ จำนวน 7 ขวดท...,"[ซื้อ, นม, เมจิ, พาสเจอร์ไรส์, จำนวน, ขวด, ปกต..."
18,https://pantip.com/topic/38412845,5fbb5a418e8105c375c6e337,{'เมจิ'},"{'นม', 'เวย์โปรตีน'}",เข้าเซเว่นเห็นนม เวย์โปรตีน เมจิ ขวด49บาทโปรตี...,"[เซเว่น, นม, เวย์โปรตีน, เมจิ, ขวด, โปรตีน, แค..."


In [25]:
#df_urls_milk.to_excel("Docs/วิเคราะห์ข้อมูล/domain.xlsx",index=False)

### ทำ Topic Modeling และคัดเลือกกระทู้ที่มีความเกี่ยวข้อง แล้วดึงคำไปใช้

In [26]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [27]:
df_join_url = pd.merge(df_cmt_process,df_urls_milk,how='inner',on='URLs')
df_join_url = df_join_url[['Retrived_date','URLs','commentId','mention_brand', 'mention_product','token_text_x']]
df_join_url.columns = ['Retrived_date','URLs','commentId','mention_brand', 'mention_product','token_text']

In [28]:
df_cmt_process.shape

(60545, 7)

In [29]:
df_join_url.tail()

,Retrived_date,URLs,commentId,mention_brand,mention_product,token_text
7851,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65703,{'โฟร์โมสต์'},"{'นม', 'รสจืด'}","[ปกติ, ขับ, ออกจาก, แยก, ไม่ได้, เลี้ยว, ไม่, ..."
7852,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65704,{'โฟร์โมสต์'},"{'นม', 'รสจืด'}","[เสียใจ, รถ, ถนน, , ไม่, , ระวัง, ตัวเอง, ดีกว..."
7853,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65705,{'โฟร์โมสต์'},"{'นม', 'รสจืด'}","[หน้า, ห้าง, ..., วาง, กรวย, หา, แพะ]"
7854,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65706,{'โฟร์โมสต์'},"{'นม', 'รสจืด'}","[ขี่, รถ, ผ่านหน้า, big c, รัชดา, ผม, วิ่ง, เล..."
7855,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65707,{'โฟร์โมสต์'},"{'นม', 'รสจืด'}","[ชีวิต, เลือก, ได้, ไม่, ขับ, มอไซด์, เสี่ยง, ..."


In [30]:
df_view = df_join_url[['commentId','mention_brand']]
df_view.column = ['commentId','mention_brand']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [31]:
df_view_count = df_view.groupby('mention_brand').count()

In [32]:
print('จำนวน comment ในกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ =',df_view.shape[0])

จำนวน comment ในกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ = 7856


In [33]:
df_view_count.sort_values(by='commentId',ascending=False)

,commentId
mention_brand,
{'เมจิ'},3395
{'ดัชมิลล์'},1341
{'โฟร์โมสต์'},1059
"{'เมจิ', 'ดัชมิลล์'}",514
{'โชคชัย'},234
{'ไทยเดนมาร์ค'},137
"{'เมจิ', 'โฟร์โมสต์'}",116
"{'หนองโพ', 'โฟร์โมสต์', 'ไทยเดนมาร์ค', 'ดัชมิลล์'}",110
"{'หนองโพ', 'โชคชัย', 'ดัชมิลล์', 'โฟร์โมสต์'}",86


In [34]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text'])

In [35]:
#dictionary.token2id.keys()

In [36]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in df_join_url['token_text']]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

<b>Domain ที่พบหลักๆ นอกเหนือจากตัว Product ห้างที่ขาย แบรนด์ (อย่างไรก็ตามต้องทำ TSNE อีกรอบเพื่อจัดกลุ่มใหม่หลังคัดเลือก Topic/Term)</b><br>
ค้นหาเทียบเคียงกับ ปัจจัยที่มีผลต่อการบริโภคนม
1. แม่และเด็ก (Kurajdova et al, 2015)
2. ออกกำลังกาย เล่นเวท วิ่ง พูดถึง Whey หางนม โปรตีน เป็นหลัก (คิดว่าแตกจากหมวดสุขภาพ)
3. โปรโมชั่น เช่น จัดโปร ส่วนลด พันแถม แพ๊กคู่ (สุภชาติ ชัยณรงค์สิงห์, 2539)
4. เอานมให้หมาแมวกิน (เทรนด์ใหม่)
5. รสชาติและคุณภาพ เช่น หอม เหม็น มัน อร่อย บูด เสีย GMP (ชาลิสา สถีระกานนท์, 2560)
6. ความยากง่ายในการหาซื้อ (Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
7. ทำกาแฟ ทำขนม (Kurajdova et al, 2015)
8. สุขภาพ ความสูง ส่วนผสมเพื่อสุขภาพ การขับถ่าย อนุมูลอิสระ (Kurajdova et al, 2015 & Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
9. ราคาและความคุ้มค่า (ปิยฉัตร ช่างเหล็ก, 2561)
<br><br>ตรงนี้ต้องหา Paper มาช่วยยันในบางหัวข้อ ถ้ามีส่วนเกินถือว่าเป็นบริบทของประเทศไทย

In [39]:
num_topics = 9
chunksize = 4000                  # size of the doc looked at every pass
iterations = 50
eval_every = 1                    # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize \
                                     ,alpha='auto', eta='auto',iterations=iterations \
                                     ,num_topics=num_topics \
                                     , eval_every=eval_every)

Wall time: 13.6 s


In [40]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, R=40, lambda_step=0.1)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.027622  0.022347       1        1  34.621532
0     -0.014512  0.005907       2        1  15.562630
3     -0.057856 -0.023575       3        1  10.658621
8     -0.002429  0.040782       4        1   9.926839
6     -0.034059 -0.079439       5        1   9.869904
4     -0.007552  0.060428       6        1   7.255671
2     -0.011340 -0.000151       7        1   4.899311
7      0.112049 -0.024351       8        1   3.724586
5      0.043321 -0.001948       9        1   3.480904, topic_info=     Category         Freq     Term        Total  loglift  logprob
23    Default  5900.000000           5900.000000  40.0000  40.0000
21    Default  4351.000000      ได้  4351.000000  39.0000  39.0000
83    Default  1027.000000   ขอบคุณ  1027.000000  38.0000  38.0000
37    Default  7867.000000      ไม่  7867.000000  37.0000  37.0000
26    Default  4282.000000      กิน  4282.000000  36.0000  36.0000
6     Default  3102.000000       นม  3102.000000  35.0000  35.0000
124   Default  2484.000000       ผม  2484.000000  34.0000  34.0000
230   Default  1075.000000      ลอง  1075.000000  33.0000  33.0000
5     Default  1948.000000       คน  1948.000000  32.0000  32.0000
57    Default  1701.000000      ชอบ  1701.000000  31.0000  31.0000
24    Default   922.000000        .   922.000000  30.0000  30.0000
33    Default  1483.000000       ทำ  1483.000000  29.0000  29.0000
161   Default  1225.000000       ดู  1225.000000  28.0000  28.0000
172   Default  1251.000000    อร่อย  1251.000000  27.0000  27.0000
327   Default   667.000000       น.   667.000000  26.0000  26.0000
69    Default  1097.000000     เวลา  1097.000000  25.0000  25.0000
321   Default   556.000000  ข้อความ   556.000000  24.0000  24.0000
104   Default  1666.000000       ดี  1666.000000  23.0000  23.0000
44    Default   732.000000   ยี่ห้อ   732.000000  22.0000  22.0000
30    Default  1956.000000     ซื้อ  1956.000000  21.0000  21.0000
2105  Default   226.000000       ฮะ   226.000000  20.0000  20.0000
303   Default  1207.000000      ใส่  1207.000000  19.0000  19.0000
508   Default   677.000000      ...   677.000000  18.0000  18.0000
11    Default   809.000000       รส   809.000000  17.0000  17.0000
1913  Default   394.000000    รีวิว   394.000000  16.0000  16.0000
436   Default   687.000000     น้อง   687.000000  15.0000  15.0000
251   Default   704.000000    กล่อง   704.000000  14.0000  14.0000
1830  Default   229.000000      หมี   229.000000  13.0000  13.0000
229   Default   920.000000      ทาน   920.000000  12.0000  12.0000
16    Default   313.000000   เซเว่น   313.000000  11.0000  11.0000
...       ...          ...      ...          ...      ...      ...
172    Topic9    47.363647    อร่อย  1251.921631   0.0833  -5.2285
33     Topic9    49.514988       ทำ  1483.505371  -0.0420  -5.1841
73     Topic9    27.810062   กระทู้   402.157959   0.6864  -5.7610
83     Topic9    40.166515   ขอบคุณ  1027.080688   0.1164  -5.3933
13     Topic9    31.872425       หา   585.645935   0.4469  -5.6246
276    Topic9    14.860615       กา   100.680038   1.4446  -6.3877
331    Topic9    22.685898      รูป   294.554871   0.7942  -5.9646
436    Topic9    30.891003     น้อง   687.943115   0.2546  -5.6559
230    Topic9    33.116325      ลอง  1075.881958  -0.1230  -5.5863
30     Topic9    39.522011     ซื้อ  1956.791260  -0.5443  -5.4095
77     Topic9    22.646641       สี   316.828247   0.7195  -5.9664
74     Topic9    28.374668      ตอน   676.118835   0.1870  -5.7409
212    Topic9    27.927395   เรื่อง   738.688477   0.0826  -5.7568
69     Topic9    29.721436     เวลา  1097.754028  -0.2513  -5.6945
314    Topic9    15.866057       ผง   137.406464   1.1991  -6.3222
161    Topic9    28.804735       ดู  1225.183838  -0.3924  -5.7258
105    Topic9    24.841404      ตัว   736.366455  -0.0313  -5.8739
18     Topic9    24.296680     เมจิ   667.198120   0.0451  -5.8960
180    Topic9

## สร้าง Bag of word dataframe<br>
ตัดคำแล้วทำ LDA อีกรอบเพื่อดู feature ที่ชัดเจนขึ้น ก่อนสร้าง Bag of word เพื่อออกกราฟ & MDS

### Create co-occurrence matrix จาก Bag of word

### Create co-occurrence matrix with Lift normalization